In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
c2007 = pd.read_csv('./CASF-2007-Affinity.csv', delimiter=' ',  header=None)
c2013 = pd.read_csv('./CASF-2013-Affinity.csv', delimiter=' ',  header=None)
c2016 = pd.read_csv('./CASF-2016-Affinity.csv', delimiter=' ',  header=None)

In [ ]:
combined_core = pd.concat([ c2016,  c2013,c2007], ignore_index=True)
combined_core.reset_index(drop=True, inplace=True)
combined_core.rename(columns={0:'id', 1:'lite', 2:'kcal/mol', 3:'yeet'}, inplace=True)
combined_core.drop(['lite','yeet'], axis = 1, inplace=True)
combined_core.drop_duplicates(subset='id',inplace=True)

In [ ]:
general = pd.read_csv('./general-2019-Affinity.csv', delimiter= ' ', header=None)
general.reset_index(drop=True, inplace=True)
general.rename(columns={0:'id', 1:'lite', 2:'kcal/mol', 3:'yeet'}, inplace=True)
general.drop(['lite','yeet'], axis = 1, inplace=True)
general.drop_duplicates(subset='id',inplace=True)

In [ ]:
refined = pd.read_csv('./refined-2019-Affinity.csv', delimiter= ' ', header=None)
refined.reset_index(drop=True, inplace=True)
refined.rename(columns={0:'id', 1:'lite', 2:'kcal/mol', 3:'yeet'}, inplace=True)
refined.drop(['lite','yeet'], axis = 1, inplace=True)
refined.drop_duplicates(subset='id',inplace=True)


In [ ]:
import math
T=295
R=1.9872*1e-3
def calc_logp(val):
    return -np.log10(np.exp(val/(T*R)))

In [ ]:
refined['pk'] = refined.apply(lambda row: calc_logp(float(row['kcal/mol'])), axis =1)
general['pk'] = general.apply(lambda row: calc_logp(float(row['kcal/mol'])), axis =1)
combined_core['pk'] = combined_core.apply(lambda row: calc_logp(float(row['kcal/mol'])), axis =1)

In [ ]:
full = pd.concat([refined, general])

In [ ]:
full.hist('pk', bins=100)

In [ ]:
# refined = refined[refined['pk'] > -25]
# general = general[general['pk'] > -25]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
combined_core.hist()

In [ ]:
general.hist(bins=30)

In [ ]:
refined.hist(bins=30)

In [ ]:
len(general) + len(refined)

In [ ]:
general_og = pd.read_csv('./general-og-values.csv',sep=r"[ ]{2,}")
refined_og = pd.read_csv('./refined-og-values.csv',sep=r"[ ]{2,}")

In [ ]:
general_og = general_og[['PDBcode', '-logKd/Ki']]
general_og.rename(columns={'PDBcode':'id','-logKd/Ki':'pk' }, inplace=True)

refined_og = refined_og[['PDBcode', '-logKd/Ki']]
refined_og.rename(columns={'PDBcode':'id','-logKd/Ki':'pk' }, inplace=True)


In [ ]:
general_og

In [ ]:
i = 0
for key in combined_core['id'].values:
    if key not in refined['id'].values and key not in general['id'].values :
        i += 1
print(i)

In [ ]:
c2007 = pd.read_csv('./CASF-2007-og.csv', sep=r"[ ]{2,}")
c2013 = pd.read_csv('./CASF-2013-og.csv', sep=r"[ ]{2,}")
c2016 = pd.read_csv('./CASF-2016-og.csv', sep=r"[ ]{2,}")

In [ ]:
c2007 = c2007[['PDBcode','pKd(pKi)']]
c2007.rename(columns={'PDBcode':'id','pKd(pKi)':'pk' }, inplace=True)
c2013 = c2013[['PDBcode','-logKd/Ki']]
c2013.rename(columns={'PDBcode':'id','-logKd/Ki':'pk' }, inplace=True)
c2016 = c2016[['PDBcode','logKa']]
c2016.rename(columns={'PDBcode':'id','logKa':'pk' }, inplace=True)

In [ ]:
combined_core_og = pd.concat([c2007, c2013, c2016], ignore_index=True)
combined_core_og.drop_duplicates(subset='id', inplace=True)

In [ ]:
# combined_core_og.to_csv('/scratch/kanakala.ganesh/ALL-CORE-affinities.csv')

In [ ]:
core_keys = combined_core['id'].values

In [ ]:
# l=[]
# for key in combined_core_og['id'].values:
#     if key in refined_og['id'].values :
#         print(key)
#         print(combined_core_og[combined_core_og['id'] == key]['pk'].item())
#         print(refined_og[refined_og['id'] == key]['pk'].item())
     

In [ ]:
final_core = []
i = 0
for key in core_keys:
    try:
        score = refined[refined['id'] == key]['pk'].item()
        true = combined_core_og[combined_core_og['id'] == key]['pk'].item()
        final_core.append([score, true])
    except:
        try:
            score = general[general['id'] == key]['pk'].item()
            true = combined_core_og[combined_core_og['id'] == key]['pk'].item()
            final_core.append([score, true]) 
        except:
            i += 1
print(i)
final_core_df = pd.DataFrame(np.array(final_core), columns=['Score', 'True'])

In [ ]:
general_keys = general_og['id'].values

In [ ]:
final_general = []
i = 0
for key in general_keys:
    try:
        score = general[general['id'] == key]['pk'].item()
        true = general_og[general_og['id'] == key]['pk'].item()
        final_general.append([score, true])
    except:
        i += 1
print(i)
final_general_df = pd.DataFrame(np.array(final_general), columns=['Score', 'True'])

In [ ]:
np.array(final_general_df['Score'].values, dtype=float)

In [ ]:
# refined_keys = refined['id'].values[refined_key_ids]
refined_keys = refined_og['id'].values

In [ ]:
final_refined = []
i = 0

for key in refined_keys:
    try:
        score = refined[refined['id'] == key]['pk'].item()
        true = general_og[general_og['id'] == key]['pk'].item()
        final_refined.append([score, true])
    except:
        i += 1
print(i)
final_refined_df = pd.DataFrame(np.array(final_refined), columns=['Score', 'True'])

In [ ]:
final_fulltrain_df = pd.concat([final_general_df, final_refined_df], ignore_index=True)

In [ ]:
final_fulltrain_df = final_fulltrain_df.sample(frac=1)

In [ ]:
from scipy.stats import pearsonr

In [ ]:
# final_fulltrain_df.to_csv('smina_score_full.csv')

In [ ]:
# final_general_df = final_general_df[final_general_df['Score'] > -10]
# final_refined_df = final_refined_df[final_refined_df['Score'] > -10]
# final_fulltrain_df = final_fulltrain_df[final_fulltrain_df['Score'] > -10]

In [ ]:
# final_core_df = final_core_df[final_core_df['Score'] > -10]

In [ ]:
len(final_general_df), len(final_refined_df), len(final_fulltrain_df)

In [ ]:
pearsonr(final_core_df['Score'].values, final_core_df['True'].values)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
math.sqrt(mean_squared_error(final_core_df['Score'].values, final_core_df['True'].values))

In [ ]:
pearsonr(np.array(final_general_df['Score'].values, dtype=float), np.array(final_general_df['True'].values, dtype=float))

In [ ]:
pearsonr(np.array(final_refined_df['Score'].values, dtype=float), np.array(final_refined_df['True'].values, dtype=float))

In [ ]:
pearsonr(np.array(final_fulltrain_df['Score'].values, dtype=float), np.array(final_fulltrain_df['True'].values, dtype=float))# fulltrain is general + refined

In [ ]:
import seaborn as sns

In [ ]:
sns.set_theme(style="ticks")

In [ ]:
# sns.lmplot(data=final_core_df, x='Score', y='True' )

In [ ]:
sns.regplot(data=final_general_df, x='Score', y='True')

In [ ]:
sns.regplot(data=final_refined_df, x='Score', y='True' )

In [ ]:
sns.regplot(data=final_fulltrain_df, x='Score', y='True')

In [ ]:
full_pdbbind_df = pd.concat([general, refined], ignore_index=True)
full_pdbbind_df.drop_duplicates(subset='id', inplace=True)

In [ ]:
full_pdbbind_df

In [ ]:
fold0 = pd.read_csv('./FOLDS/full_pk_cv_test0.types', sep = ' ')
fold1 = pd.read_csv('./FOLDS/full_pk_cv_test1.types', sep = ' ')
fold2 = pd.read_csv('./FOLDS/full_pk_cv_test2.types', sep = ' ')

In [ ]:
fold0_keys = fold0['pdbid'].values

In [ ]:
final_fold0 = []
for key in fold0_keys:
    try:
        score = full_pdbbind_df[full_pdbbind_df['id'] == key]['pk'].item()
        true = fold0[fold0['pdbid'] == key]['affinity'].item()
        
        final_fold0.append([score, true])
    except:
        print(key)
final_fold0_df = pd.DataFrame(np.array(final_fold0), columns=['Score', 'True'])

In [ ]:
fold1_keys = fold1['pdbid'].values

In [ ]:
final_fold1 = []
for key in fold1_keys:
    try:
        score = full_pdbbind_df[full_pdbbind_df['id'] == key]['pk'].item()
        true = fold1[fold1['pdbid'] == key]['affinity'].item()
        
        final_fold1.append([score, true])
    except:
        print(key)
final_fold1_df = pd.DataFrame(np.array(final_fold1), columns=['Score', 'True'])

In [ ]:
fold2_keys = fold2['pdbid'].values

In [ ]:
final_fold2 = []
for key in fold2_keys:
    try:
        score = full_pdbbind_df[full_pdbbind_df['id'] == key]['pk'].item()
        true = fold2[fold2['pdbid'] == key]['affinity'].item()
        
        final_fold2.append([score, true])
    except:
        print(key)
final_fold2_df = pd.DataFrame(np.array(final_fold2), columns=['Score', 'True'])

In [ ]:
# final_fold0_df = final_fold0_df[final_fold0_df['Score'] > -20]
# final_fold1_df = final_fold1_df[final_fold1_df['Score'] > -20]
# final_fold2_df = final_fold2_df[final_fold2_df['Score'] > -20]

In [ ]:
len(final_fold0_df), len(final_fold1_df), len(final_fold2_df)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
pearsonr(final_fold0_df['Score'].values, final_fold0_df['True'].values)

In [ ]:
pearsonr(final_fold1_df['Score'].values, final_fold1_df['True'].values)

In [ ]:
pearsonr(final_fold2_df['Score'].values, final_fold2_df['True'].values)

In [ ]:
math.sqrt(mean_squared_error(final_fold0_df['Score'].values, final_fold0_df['True'].values))

In [ ]:
math.sqrt(mean_squared_error(final_fold1_df['Score'].values, final_fold1_df['True'].values))

In [ ]:
math.sqrt(mean_squared_error(final_fold2_df['Score'].values, final_fold2_df['True'].values))

In [ ]:
sns.regplot(data=final_fold0_df, x='Score', y='True')

In [ ]:
sns.regplot(data=final_fold1_df, x='Score', y='True')

In [ ]:
sns.regplot(data=final_fold2_df, x='Score', y='True')